
# 🇮🇩 IndoBERT Base P2 — Seq2Seq Summarization (mirrors Rust pipeline)

Notebook ini **menyamakan alur** dengan kode Rust-mu:
- **Tokenizer/Model**: `indobenchmark/indobert-base-p2` dibungkus jadi **EncoderDecoder** (encoder = BERT, decoder = BERT + cross-attention).
- **Training**: Hugging Face **Trainer** dengan **Masked CE** (PAD di-ignore otomatis).
- **Evaluasi**: **ROUGE-1 / ROUGE-2 / ROUGE-L (F1)**.
- **Contoh Generasi**: 3 sampel, print teks + ROUGE per-sample.
- **Simpan model**: disimpan ke folder lokal.

> Dataset diasumsikan CSV dengan kolom `text` dan `summary` (sama seperti di Rust).


In [ ]:

# Jalankan ini kalau dependensi belum ada di environment kamu
# %pip install -q transformers datasets evaluate rouge-score accelerate sentencepiece


In [1]:

import os
import random
import numpy as np
import pandas as pd
import torch

from datasets import load_dataset, DatasetDict
from transformers import (
    AutoTokenizer, AutoConfig, EncoderDecoderModel,
    DataCollatorForSeq2Seq, Trainer, TrainingArguments
)
import evaluate

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

CSV_PATH = "Benchmark.csv"
TEXT_COL = "text"
SUMMARY_COL = "summary"

MODEL_NAME = "indobenchmark/indobert-base-p2"

MAX_SOURCE_LENGTH = 512
MAX_TARGET_LENGTH = 64
VAL_MAX_TARGET_LENGTH = 64
NUM_BEAMS = 4
MIN_NEW_TOKENS = 10
LEARNING_RATE = 5e-5
BATCH_SIZE = 4
EPOCHS = 1

device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cpu'

In [2]:

data_files = {"train": CSV_PATH}
raw_ds = load_dataset("csv", data_files=data_files)

ds_split = raw_ds["train"].train_test_split(test_size=0.1, seed=SEED)
ds = DatasetDict({"train": ds_split["train"], "test": ds_split["test"]})

print(ds)
print("Sample row:", {k: ds['train'][k][0] for k in [TEXT_COL, SUMMARY_COL]})


DatasetDict({
    train: Dataset({
        features: ['text', 'summary'],
        num_rows: 18
    })
    test: Dataset({
        features: ['text', 'summary'],
        num_rows: 2
    })
})
Sample row: {'text': 'Jakarta , CNN Indonesia - - Presiden Joko Widodo ( Jokowi ) meminta agar masyarakat Indonesia tidak saling menghujat dan menebar fitnah melalui berita yang tidak memiliki kebenaran atau hoax di media sosial . Hal tersebut menurut Jokowi membuat Indonesia menjadi tidak produktif sebagai sebuah bangsa . " Saling menyalahkan , memfitnah , membuat berita-berita hoax di media sosial itu tidak memiliki kontribusi pada negara ini , " kata Jokowi saat meresmikan SMA Negeri Taruna Nala Jawa Timur , Kecamatan Kedungkandang , Kota Malang , dikutip dari Antara , Sabtu ( 3 / 6 ) . Menurut mantan Walikota Solo , Indonesia memiliki bonus demografi sampai 2030 . Oleh karena itu tingginya jumlah angkatan kerja Indonesia harus disalurkan ke arah yang positif . Bukan malah saling bergesekan di m

In [4]:
# ===== Tokenizer & IndoBERT-as-EncoderDecoder =====
from transformers import AutoTokenizer, EncoderDecoderModel

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

# Pastikan ada PAD token; hitung berapa token baru yang ditambah
added_tokens = 0
if tokenizer.pad_token is None:
    added_tokens = tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Bungkus BERT -> Seq2Seq (encoder-decoder)
model = EncoderDecoderModel.from_encoder_decoder_pretrained(
    MODEL_NAME,  # encoder
    MODEL_NAME   # decoder (akan di-set is_decoder=True + add_cross_attention=True)
)

# Start/end/pad token selaras “gaya Rust”: start=CLS, end=SEP
decoder_start = tokenizer.cls_token_id if tokenizer.cls_token_id is not None else tokenizer.bos_token_id
eos_id        = tokenizer.sep_token_id if tokenizer.sep_token_id is not None else tokenizer.eos_token_id

model.config.decoder_start_token_id = decoder_start
model.config.eos_token_id = eos_id
model.config.pad_token_id = tokenizer.pad_token_id

# Default generasi
model.config.max_length = VAL_MAX_TARGET_LENGTH
model.config.num_beams  = NUM_BEAMS

# ❗️JANGAN: model.resize_token_embeddings(...)
# ✅ Jika ada token baru ditambahkan ke tokenizer, resize embeddings encoder & decoder secara terpisah
if added_tokens and added_tokens > 0:
    model.encoder.resize_token_embeddings(len(tokenizer))
    model.decoder.resize_token_embeddings(len(tokenizer))

# Ke device
model.to(device)

print("Loaded IndoBERT EncoderDecoder ✅")
print(f"pad/cls/sep ids: {tokenizer.pad_token_id}/{tokenizer.cls_token_id}/{tokenizer.sep_token_id}")


Some weights of BertLMHeadModel were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'encoder.layer.0.crossattention.output.LayerNorm.bias', 'encoder.layer.0.crossattention.output.LayerNorm.weight', 'encoder.layer.0.crossattention.output.dense.bias', 'encoder.layer.0.crossattention.output.dense.weight', 'encoder.layer.0.crossattention.self.key.bias', 'encoder.layer.0.crossattention.self.key.weight', 'encoder.layer.0.crossattention.self.query.bias', 'encoder.layer.0.crossattention.self.query.weight', 'encoder.layer.0.crossattention.self.value.bias', 'encoder.layer.0.crossattention.self.value.weight', 'encoder.layer.1.crossattention.output.LayerNorm.bias', 'encoder.layer.1.crossattention.output.LayerNorm.weig

Loaded IndoBERT EncoderDecoder ✅
pad/cls/sep ids: 0/2/3


In [5]:

def preprocess_function(examples):
    inputs  = examples[TEXT_COL]
    targets = examples[SUMMARY_COL]

    model_inputs = tokenizer(
        inputs,
        max_length=MAX_SOURCE_LENGTH,
        truncation=True
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=MAX_TARGET_LENGTH,
            truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_ds = ds.map(
    preprocess_function,
    batched=True,
    remove_columns=ds["train"].column_names
)
tokenized_ds


Map:   0%|          | 0/18 [00:00<?, ? examples/s]

/Users/mraffyzeidan/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 18
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2
    })
})

In [6]:

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=-100
)

rouge = evaluate.load("rouge")

def postprocess_text(preds, labels):
    preds = [p.strip() for p in preds]
    labels = [l.strip() for l in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )
    return {"rouge1": result["rouge1"], "rouge2": result["rouge2"], "rougeL": result["rougeL"]}


In [ ]:
# ===== Training (Seq2Seq) =====
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

output_dir = "outputs-indobert-p2-summarization"

args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    # >>> penting untuk evaluasi generasi
    predict_with_generate=True,
    generation_max_length=VAL_MAX_TARGET_LENGTH,
    generation_num_beams=NUM_BEAMS,
    # <<<
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,
    report_to="none",
    seed=SEED,
    fp16=torch.cuda.is_available(),  # mixed precision di GPU
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


In [ ]:

metrics = trainer.evaluate()
metrics


In [ ]:

# ===== Example Generations (3 samples) =====
def show_samples(n_samples=3):
    import random, torch
    n = min(n_samples, len(ds["test"]))
    idxs = random.sample(range(len(ds["test"])), k=n)
    for i, idx in enumerate(idxs, 1):
        item = ds["test"][idx]
        text, ref = item[TEXT_COL], item[SUMMARY_COL]

        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=MAX_SOURCE_LENGTH
        ).to(device)

        with torch.no_grad():
            gen_ids = trainer.model.generate(
                **inputs,
                max_length=VAL_MAX_TARGET_LENGTH,
                num_beams=NUM_BEAMS,
                min_new_tokens=MIN_NEW_TOKENS,
                early_stopping=True
            )

        pred = tokenizer.decode(gen_ids[0], skip_special_tokens=True)
        res = rouge.compute(predictions=[pred], references=[ref], use_stemmer=True)

        print(f"""
--- Sample {i} ---
Original (first 150 chars):
{(text[:150] + '...') if len(text) > 150 else text}

Reference:
{ref}

Generated:
{pred}

Sample ROUGE-1: {res['rouge1']:.4f}, ROUGE-2: {res['rouge2']:.4f}, ROUGE-L: {res['rougeL']:.4f}
""")

show_samples(3)


In [ ]:

save_dir = "indobert-base-p2-encoderdecoder-summarizer"
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)
print("Saved to:", save_dir)
